In [1]:
! pip install datasets

In [2]:
import os

In [3]:
! pip install datasets transformers sentencepiece sacrebleu

In [4]:
model_checkpoint="facebook/nllb-200-distilled-600M"

In [5]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.src_lang = "ory_Orya"  
tokenizer.tgt_lang = "hin_Deva"

C:\Users\Muktikanta\.conda\envs\pytorch-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
hindi_train=[]
hindi_test=[]
odia_train=[]
odia_test=[]
with open("odia_train3.0.txt",'r',encoding="utf8") as f1, open("odia_test3.0.txt",'r',encoding="utf8") as f2, open("hindi_train3.0.txt",'r',encoding="utf8") as f3, open("hindi_test3.0.txt",'r',encoding="utf8") as f4:
    for l in f1:
        line=l[:-1]
        odia_train.append(line)
    for l in f2:
        line=l[:-1]
        odia_test.append(line)
    for l in f3:
        line=l[:-1]
        hindi_train.append(line)
    for l in f4:
        line=l[:-1]
        hindi_test.append(line)

In [7]:
from datasets import load_dataset
import json

In [8]:
data=[]
for i in range(len(hindi_train)):
    c={}
    c['translation']={}
    c['translation']['ory_Orya']=odia_train[i]
    c['translation']['hin_Deva']=hindi_train[i]
    data.append(dict(c))


f1=open("train.json","a")
for i in range(len(data)):
    obj=json.dumps(data[i])
    f1.write(obj+'\n')

    
data=[]
for i in range(len(hindi_test)):
    c={}
    c['translation']={}
    c['translation']['ory_Orya']=odia_train[i]
    c['translation']['hin_Deva']=hindi_train[i]
    data.append(dict(c))

f1=open("test.json","a")
for i in range(len(data)):
    obj=json.dumps(data[i])
    f1.write(obj+'\n')

In [9]:
from datasets import load_dataset, load_metric
raw_datasets = load_dataset("json", data_files={"train":"train.json","validation":"test.json"})
metric = load_metric("sacrebleu")

Using custom data configuration default-c5ca58a6a802bfcf
C:\Users\Muktikanta\.conda\envs\pytorch-gpu\lib\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files: 100%|████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 669.70it/s]


Dataset json downloaded and prepared to C:\Users\Muktikanta\.cache\huggingface\datasets\json\default-c5ca58a6a802bfcf\0.0.0\a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 32.93it/s]


In [10]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 25770
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 11032
    })
})

In [11]:
raw_datasets["validation"][0]

{'translation': {'ory_Orya': 'ପଞ୍ଜାବ-ହରିୟାଣା ସମେତ ଅନେକ ରାଜ୍ୟର କୃଷକମାନେ ଜାତୀୟ ରାଜଧାନୀ ଦିଲ୍ଲୀର ସୀମାରେ କେନ୍ଦ୍ର ସରକାରଙ୍କ ନୂତନ କୃଷି ନିୟମକୁ ବିରୋଧ କରୁଛନ୍ତି।',
  'hin_Deva': 'केंद्र सरकार के नए कृषि कानूनों को लेकर पंजाब-हरियाणा समेत कई राज्यों के किसान राष्ट्रीय राजधानी दिल्ली की सीमाओं पर धरना प्रदर्शन कर रहे हैं ।'}}

In [12]:
if model_checkpoint == "od-hi-nllb-fine-tune-checkpoint":
    prefix = "translate Odia to Hindi: "
else:
    prefix = ""


In [13]:
max_input_length = 128
max_target_length = 128
source_lang = "ory_Orya"
target_lang = "hin_Deva"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x0000014C315FDD30> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
  0%|                                                                                           | 0/26 [00:00<?, ?ba/s]C:\Users\Muktikanta\.conda\envs\pytorch-gpu\lib\site-packages\transformers\tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts 

In [15]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [16]:
!nvidia-smi

Mon Feb 12 16:21:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.96                 Driver Version: 536.96       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro RTX 8000              WDDM  | 00000000:17:00.0 Off |                  Off |
| 33%   32C    P8              14W / 260W |   1607MiB / 49152MiB |     11%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
batch_size = 38
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=60,
    predict_with_generate=True,
    fp16=True
    
)

In [18]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [19]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [20]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    
    
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.390200,0.988998,38.487700,31.671300
2,1.252500,0.910395,40.911500,31.642000
3,1.128800,0.848624,42.437800,31.686600
4,1.069500,0.797443,44.198800,31.636200
5,1.035700,0.750310,45.796100,31.713200
6,0.977300,0.708208,46.968600,31.722600
7,0.932700,0.671238,48.272700,31.653400
8,0.909700,0.634377,49.799800,31.646800
9,0.857700,0.602178,51.306100,31.867400
10,0.831600,0.572397,52.643000,31.838400


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation 

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}


In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


# # Define the checkpoint path
# checkpoint_path = "C:\\Users\\Muktikanta\\nllb_odi_hindi\\nllb-200-distilled-600M-finetuned-ory_Orya-to-hin_Deva\\checkpoint-12500"

# # Load the trained model checkpoint
# model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

import pickle
pickle.dump(model, open('model.pkl','wb'))
pickle.dump(tokenizer, open('tokenizer.pkl','wb'))
# import pickle

# Load the model and tokenizer
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# Prompt the user to type a sentence
input_sentence = input("Enter a sentence to translate: ")

# Translate the input sentence
input_ids = tokenizer.encode(input_sentence, return_tensors="pt")
translated_ids = model.generate(input_ids)
translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)

# Print the translated text
print("Translated sentence:", translated_text)


# with open('model.pkl', 'rb') as f:
#     model = pickle.load(f)
# with open('tokenizer.pkl', 'rb') as f:
#     tokenizer = pickle.load(f)
# # Input file path containing sentences to be translated
# input_file_path = "C:\\Users\\Muktikanta\\nllb_distilled-600M-odia-hindi\\1-10_sentence.txt"

# # Output file path to save the translated sentences
# output_file_path = "C:\\Users\\Muktikanta\\nllb_distilled-600M-odia-hindi\\model_translated_text.txt"

# # Read sentences from the input file
# with open(input_file_path, "r", encoding="utf-8") as input_file:
#     input_sentences = input_file.readlines()

# # Translate each input sentence and save the translations to the output file
# with open(output_file_path, "w", encoding="utf-8") as output_file:
#     for input_sentence in input_sentences:
#         input_sentence = input_sentence.strip()
#         input_ids = tokenizer.encode(input_sentence, return_tensors="pt")
#         translated_ids = model.generate(input_ids)
#         translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
#         output_file.write(translated_text + "\n")

# print("Translation complete. Translated sentences saved to:", output_file_path)

Enter a sentence to translate: ପ୍ରାରମ୍ଭିକ ଦିନରେ ଅଛି।
Translated sentence: शुरुआती दिनों में है।


In [ ]:
def translate_text(source_text):
    # Replace this with your translation logic using the loaded model
    input_ids = tokenizer.encode(source_text, return_tensors="pt")
    translated_ids = translation_model.generate(input_ids)
    translated_token = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    translated_text = tokenizer.translate(source_text)
    return translated_text

In [1]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.


In [10]:
import transformers

print(transformers.__version__)

4.21.2


In [7]:
!pip install --upgrade transformers





  Using cached transformers-4.37.2-py3-none-any.whl (8.4 MB)
     ---------------------------------------- 2.2/2.2 MB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 269.7/269.7 kB ? eta 0:00:00
  Using cached huggingface_hub-0.20.3-py3-none-any.whl (330 kB)
  Using cached fsspec-2024.2.0-py3-none-any.whl (170 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.7.1
    Uninstalling fsspec-2022.7.1:
      Successfully uninstalled fsspec-2022.7.1
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.9.1
    Uninstalling huggingface-hub-0.9.1:
      Successfully uninstalled huggingface-hub-0.9.1
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.21.2
    Uninstalling transformers-4.21.2:
      Successfull

In [11]:
import transformers
print(transformers.__version__)


4.21.2


In [13]:
!pip install tensorflow


     -------------------------------------- 300.8/300.8 MB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 47.6 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 92.3 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 34.6 MB/s eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 39.2 MB/s eta 0:00:00
     ---------------------------------------- 442.0/442.0 kB ? eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB ? eta 0:00:00
     --------------------------------------- 15.8/15.8 MB 40.9 MB/s eta 0:00:00
     --------------------------------------- 24.4/24.4 MB 26.2 MB/s eta 0:00:00
     -------------------------------------- 938.4/938.4 kB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 18.1 MB/s eta 0:00:00
     ---------------------------------------- 133.7/133.7 kB ? e

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Muktikanta\\.conda\\envs\\pytorch-gpu\\Lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




     ---------------------------------------- 103.9/103.9 kB ? eta 0:00:00
     ------------------------------------- 181.3/181.3 kB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 84.9/84.9 kB ? eta 0:00:00
     -------------------------------------- 151.7/151.7 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.1
    Uninstalling protobuf-3.20.1:
      Successfully uninstalled protobuf-3.20.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.1
    Uninstalling numpy-1.23.1:
      Successfully uninstalled numpy-1.23.1


In [17]:
pip check


spyder 5.2.2 requires pyqtwebengine, which is not installed.Note: you may need to restart the kernel to use updated packages.
spyder 5.2.2 has requirement pyqt5<5.13, but you have pyqt5 5.15.7.
scipy 1.9.3 has requirement numpy<1.26.0,>=1.18.5, but you have numpy 1.26.4.



Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "deptree"

